In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import math

import tikzplotlib

from matplotlib.pyplot import figure

from scipy import integrate

In [ ]:
precision = 0.0000001

# Selections of specified measure, $\mu = 0.4$ in the paper (Fig. 5)

In [ ]:
my_area_to_select = 0.4

In [ ]:
def samesign(a, b):
        return a * b > 0

def invert(funct, value, low, high):
    def bisect(func, low, high):
        'Find root of continuous function where f(low) and f(high) have opposite signs'
        assert not samesign(func(low), func(high))
        midpoint =0
        while abs(low-high) > precision:
            midpoint = (low + high) / 2.0
            if samesign(func(low), func(midpoint)):
                low = midpoint
            else:
                high = midpoint
        return midpoint
    def to_bissect(x):
        return funct(x)-value
    return bisect(to_bissect, low, high)

def min_k(area):
    return invert(lambda k: 2*k-k*k, area, 0, 1)

def max_k(area):
    return invert(lambda k: k*k, area, 0, 1)

def sum_k(area):
    return invert(lambda k: sum_area(k), area, 0, 1)

def prod_k(area):
    return invert(lambda k: prod_area(k), area, 0, 1)

def sum_area(k):
    if k<1:
        return 0.5*k*k
    else:
        return 0.5*k*k - (k-1)*(k-1)
    
def prod_area(k):
    if k == 0:
        return 0.0
    else:
        return k - k * math.log(k)
    
def min_x2(x1, k):
    if x1 is None:
        return np.nan
    if x1 < k:
        return np.nan
    return k

def max_x2(x1, k):
    if x1 is None:
        return np.nan
    if x1 > k:
        return np.nan
    return k

def sum_x2(x1, k):
    if x1 is None:
        return np.nan
    if k-x1>1:
        return np.nan
    if x1 > k: 
        return np.nan
    return k-x1

def prod_x2(x1, k):
    if x1 is None:
        return np.nan
    if x1 < k:
        return np.nan
    return k/x1

In [ ]:
my_min_k = min_k(my_area_to_select)
my_max_k = max_k(my_area_to_select)
my_sum_k = sum_k(my_area_to_select)
my_prod_k = prod_k(my_area_to_select)
#print("my_uniform_k: "+str(my_uniform_k))

x1_values =  np.linspace(0.0, 1, 1000)

min_x2_values = np.array(list(map(lambda x1: min_x2(x1, my_min_k), x1_values)))
max_x2_values = np.array(list(map(lambda x1: max_x2(x1, my_max_k), x1_values)))
sum_x2_values = np.array(list(map(lambda x1: sum_x2(x1, my_sum_k), x1_values)))
prod_x2_values = np.array(list(map(lambda x1: prod_x2(x1, my_prod_k), x1_values)))

# plotting the line 1 points
figure(figsize=(4, 4), dpi=80)
plt.plot(x1_values, min_x2_values, "k-",label = "$min(x_1,x_2)$") 
plt.plot(min_x2_values, x1_values, "k-", label='_nolegend_') 
plt.plot(x1_values, max_x2_values, "k:",label = "$max(x_1,x_2)$") 
plt.plot( max_x2_values, x1_values, "k:", label='_nolegend_') 
plt.plot(x1_values, sum_x2_values, "k-.",label = "$(x_1 + x_2)/2$") 
plt.plot(x1_values, prod_x2_values, "k--",label = "$x_1 x_2$ (po)") 
plt.xlim(0, 1)
plt.ylim(0, 1) 
# naming the x axis 
plt.xlabel('$x_1$') 
# naming the y axis 
plt.ylabel('$x_2$') 
# giving a title to my graph 
#plt.title('impartiality for various utility functions!') 
  
# show a legend on the plot 
plt.legend() 

plt.grid(True)
  
# function to show the plot 
#plt.savefig('selections_40pct.png')
#tikzplotlib.save("selections_40pct.tex")

plt.show() 

# Diversity vs area (Fig. 6)

In [ ]:
def prod_choice(k):
    return (k-k*math.log(k))*(k-k*math.log(k))-k*k*(0.5-math.log(k))

def prod_diversity(area): 
    if area==0: 
        return 1
    else:
        k = invert(lambda k: prod_area(k), area, 0, 1) 
        return prod_choice(k)/(area*area)

def sum_po_integral(k):
    if k <= 1:
        return (1/24)*k*k*k*k
    elif k>1:
        return (1/4)*(k-1)*(k-1) + 1/8 -k/3+(k*k)/4 -(1/8)*(k-1)**4 +(1/3)*k*(k-1)**3 - (1/4)*k*k*(k-1)*(k-1)

def sum_diversity(area): 
    if area==0: 
        return 2/3
    else:
        k = invert(lambda k: sum_area(k), area, 0, 2) 
        return (area*area-2*sum_po_integral(k))/(area*area)


def max_diversity(area):   
    return 0.5


def min_area(k):
    return 2*k-k*k


def min_po_integral(k):
    return k*(k/4)*(2-k*k)


def min_diversity(area):    
    if area==0: 
        return 3/4
    else:
        k = invert(lambda k: min_area(k), area, 0, 1) 
        return (area*area-2*min_po_integral(k))/(area*area)

In [ ]:
areas =  np.linspace(0.0, 1, 100)  

min_diversities = np.array(list(map(lambda area: min_diversity(area), areas)))
max_diversities = np.array(list(map(lambda area: max_diversity(area), areas)))
sum_diversities = np.array(list(map(lambda area: sum_diversity(area), areas)))
prod_diversities = np.array(list(map(lambda area: prod_diversity(area), areas)))

figure(figsize=(4, 4), dpi=80)
# plotting the line 1 points
# default: matplotlib.rcParams['figure.figsize'] = [8.0, 6.0]
plt.plot(areas, min_diversities, "k-",label = "$min(x_1,x_2)$") 
plt.plot(areas, max_diversities, "k:",label = "$max(x_1,x_2)$") 
plt.plot(areas, sum_diversities, "k-.",label = "$(x_1+x_2)/2$") 
plt.plot(areas, prod_diversities, "k--",label = "$x_1 x_2$ (po)") 

plt.xlim(0, 1)
plt.ylim(0, 1) 
# naming the x axis 
plt.xlabel('percentage selected') 
# naming the y axis 
plt.ylabel('diversity') 
  
# show a legend on the plot 
plt.legend(loc='lower right') 

plt.grid(True)
  
# function to show the plot 
#plt.savefig('diversity_vs_area.png')
#tikzplotlib.save("diversity_vs_area.tex")

plt.show() 
    

# Rarefication example (Fig. 8-a)

In [ ]:
my_area_to_select = 0.1

In [ ]:
def uniform_area(k):
    if k == 0:
        return 0.0
    else:
        return k - k * math.log(k)

#def rarified_xi_area(k):
#    return 2*math.sqrt(k)-k 
def rarified_xi_area(k):
    if k == 0:
        return 0.0
    else:
        return k - k * math.log(k)

#def rarified_x1x2_area(k):
#    return k+2*math.sqrt(k)*(1-math.sqrt(k)) 
def rarified_x1x2_area(k):
    if k == 0:
        return 0.0
    else:
         return k - k * math.log(k)

def uniform_k(area):
    return invert(lambda k: uniform_area(k), area, 0, 1)
    
def rarified_xi_k(area):
    return invert(lambda k:  rarified_xi_area(k), area, 0, 1)

def rarified_x1x2_k(area):
    return invert(lambda k:  rarified_x1x2_area(k), area, 0, 1)
    

def uniform_x2(x1, k):
    if x1 is None:
        return np.nan
    if x1 <= k:
        return np.nan
    return k/x1

def rarified_x2_x2(x1, k):
    if x1 <= k:
        return np.nan
    return math.sqrt(k/x1)

def rarified_x1x2_x2(x1, k):
    if x1 <= math.sqrt(k):
        return np.nan
    return math.sqrt(k)/x1

In [ ]:
my_uniform_k = uniform_k(my_area_to_select)
#print("my_uniform_k: "+str(my_uniform_k))
my_rarified_xi_k =  rarified_xi_k(my_area_to_select)
my_rarified_x1x2_k =  rarified_x1x2_k(my_area_to_select)

x1_values =  np.linspace(0.0, 1, 1000)

uniform_x2_values = np.array(list(map(lambda x1: uniform_x2(x1, my_uniform_k), x1_values)))
rarified_x2_x2_values = np.array(list(map(lambda x1: rarified_x2_x2(x1, my_rarified_xi_k), x1_values)))
rarified_x1x2_x2_values = np.array(list(map(lambda x1: rarified_x1x2_x2(x1, my_rarified_x1x2_k), x1_values)))

In [ ]:
figure(figsize=(4, 4), dpi=80)
# plotting the line 1 points  
plt.plot(x1_values, uniform_x2_values, "k-",label = "$dx_1 dx_2$") 
plt.plot(x1_values, rarified_x2_x2_values, "k:",label = "$2x_2 dx_1 dx_2$") 
plt.plot(x1_values, rarified_x1x2_x2_values, "k-.",label = "$4x_1 x_2 dx_1 dx_2$") 
plt.xlim(0, 1)
plt.ylim(0, 1) 
# naming the x axis 
plt.xlabel('$x_1$') 
# naming the y axis 
plt.ylabel('$x_2$') 
# giving a title to my graph 
#plt.title('impartiality for various utility functions!') 
  
# show a legend on the plot 
plt.legend() 
plt.grid(True)
  
# function to show the plot 
#plt.savefig('rarefication_dx1_dx2.png')
# saving in tikz formal
#tikzplotlib.save('rarefication_dx1_dx2.tex')    
plt.show()   

# Cone relation & trade-off (Fig. 8-b)

In [ ]:
my_area_to_select = 0.25
# a here is the inverse of the a in the paper
my_tradeoff_a =[0.5, 2, 10]
my_linestyle=["k:","k-.","k--"]
my_linelabel=["a=2","a=0.5","a=0.1"]
precision = 0.0000001

In [ ]:
def tradeoff_y(x, a, k):
    triangle=0.5*a*(1-x)*(1-x)
    rectangle = x*a*(1-x)
    if rectangle+triangle>=k:
        return min(max(0,-a*x+math.sqrt(a*a*x*x+2*a*k)),1)
    else:
        return min(max(0,k+triangle),1)
                       
def tradeoff_area(a,k):
    integral, _ = integrate.quad(lambda x: tradeoff_y(x, a, k), 0, 1)
    return integral

def tradeoff_k(area, a):
    return invert(lambda k: tradeoff_area(a,k), area, 0, 1)

def tradeoff_x2_values(x1, a, k):
    float_result = tradeoff_y(x1, a, k)
    if float_result <= 0 or float_result>=1:
        return np.nan
    else:
        return float_result

In [ ]:
my_tradeoff_k = []
for a in my_tradeoff_a:
    my_tradeoff_k.append(tradeoff_k(my_area_to_select, a))
    
x1_values =  np.linspace(0.0, 1, 1000)
my_tradeoff_x2_values=[]
for i in range(len(my_tradeoff_a)):
    my_tradeoff_x2_values.append(np.array(list(map(lambda x1: tradeoff_x2_values(x1, my_tradeoff_a[i], my_tradeoff_k[i]), x1_values))))

figure(figsize=(4, 4), dpi=80)
# plotting the line 1 points  
for i in range(len(my_tradeoff_a)):
    plt.plot(x1_values, my_tradeoff_x2_values[i], my_linestyle[i],label ='${}$'.format(my_linelabel[i])) 
plt.xlim(0, 1)
plt.ylim(0, 1) 
# naming the x axis 
plt.xlabel('$x_1$') 
# naming the y axis 
plt.ylabel('$x_2$') 
# giving a title to my graph 
#plt.title('impartiality for various utility functions!') 
plt.grid(True)
  
# show a legend on the plot 
plt.legend() 
  
# function to show the plot 
#plt.savefig('tradeoff.png')
#tikzplotlib.save('tradeoff.tex') 
plt.show() 

# Sorting by Pareto fronts vs by $k$-Pareto optimality

## Selections of measure $\mu=0.2$, Fig. 12

The set $X$ is composed of a large number of points placed on a regular grid.

In [ ]:
def whisker_y(x, k):
    def f_A(x, k):
        return 0.5-x
    def f_B(x, k):
        return -x+math.sqrt(2*k)+0.5
    def f_C(x, k):
        return -0.5*x+(k/x)+0.5
    def f_D(x, k):
        return math.sqrt((x-0.5)*(x-0.5)+2*k)+0.5-x
    if k ==0:
        return f_A(x, k)
    else:
        if x > 0.5: 
            return f_D(x, k)
        elif x > math.sqrt(2*k): 
            return f_B(x, k)
        elif x>k:
            return f_C(x, k)
        else:
            return 1
                      
def whisker_area(k):
    integral, _ = integrate.quad(lambda x: whisker_y(x, k), 0, 1)
    return integral - 0.125

def whisker_k(area):
    return invert(lambda k: whisker_area(k), area, 0.0001, 0.125)
         

def whisker_x2_values(x1, k):
    float_result = whisker_y(x1, k)
    if float_result <= 0 or float_result>=1:
        return np.nan
    else:
        return float_result

def flip(np_array):
     return np_array
#    return 1-np_array

def reflip(np_array):
     return 1-np_array
#    return np_array
     
def l_bound(x):
    if x <0.5:
        return 0.75-0.5*x
    elif x<0.75:
        return 0.5-2*(x-0.5)
    else:
        return 0
    
def u_bound(x):
    if x<0.5:
        return 1
    else:
        return 1-(x-0.5)

In [ ]:
my_area_to_select = 0.1 #selecting 20 percent of the shaded area
# a here is the inverse of the a in the paper
whisker_linestyle="k-"
whisker_linelabel="po"
nsgaii_linestyle="k-."
limit_linestyle="k-"
nsgaii_linelabel="Pareto fronts"
precision = 0.0000001

In [ ]:
my_whisker_k = whisker_k(my_area_to_select)
    
x1_values =  np.linspace(0.0, 1, 2500)
    
my_whisker_x2_values= np.array(list(map(lambda x1: whisker_x2_values(x1, my_whisker_k), x1_values)))
my_nsgaii_x2_values= np.array(list(map(lambda x1: math.sqrt(2*my_area_to_select+0.25)-x1, x1_values)))
u_bound_x2_values= np.array(list(map(lambda x1: u_bound(x1), x1_values)))
l_bound_x2_values= np.array(list(map(lambda x1: l_bound(x1), x1_values)))

figure(figsize=(4, 4), dpi=80)
plt.plot( flip(x1_values), flip(my_whisker_x2_values),
         whisker_linestyle,label =whisker_linelabel) 
plt.plot(flip(x1_values), flip(my_nsgaii_x2_values), 
         nsgaii_linestyle,label =nsgaii_linelabel) 
#plt.plot(x1_values, u_bound_x2_values, limit_linestyle) 
#plt.plot(x1_values, l_bound_x2_values, limit_linestyle) 
plt.fill_between(reflip(x1_values), reflip(l_bound_x2_values), reflip(u_bound_x2_values),  hatch="....", facecolor='lightgrey',edgecolor='dimgray')
plt.plot()
#plt.axes().set_aspect('equal', adjustable='box')

plt.xlim(0, 1)
plt.ylim(0, 1) 
# naming the x axis 
plt.xlabel('$x_1$') 
# naming the y axis 
plt.ylabel('$x_2$') 
# set ticks and 
#plt.xticks([])
#plt.yticks([])
# giving a title to my graph 
#plt.title('impartiality for various utility functions!') 
  
# show a legend on the plot 
plt.legend() 
  
# function to show the plot 
# plt.savefig('whisker.png')
# tikzplotlib.save('selection_by_fronts_and_po.tex') 
plt.show() 
    

## Visualization of equivalence classes (fronts)

We'll use implementation of Nondominated sorting and sorting by $po$ from `deap` library.

In [ ]:
from deap import base, creator
from deap.benchmarks.tools import hypervolume
import deap.tools as tools
from deap.tools.emo import *
#import random

In [ ]:
def eval_func(ind):
    return tuple(ind)

creator.create("FitnessMax", base.Fitness, weights=(-1.0,) * 2)
# every individual is a list
creator.create("Individual", list, fitness=creator.FitnessMax)
    
# toolbox initialization
toolbox = base.Toolbox()

toolbox.register("attr_bool", np.random.rand) #random.rand, 0, 1)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", eval_func)

In [ ]:
def create_uniform_grid(low=0, high=1, num=10):
    pop = []
    delta = (high-low) / num
    for i in range(0, num + 1):
        for j in range(0, num + 1):
            pop.append(creator.Individual([np.round(low + i*delta, 3),np.round(low + j*delta, 3)]))
    return pop

In [ ]:
def calc_fitness(pop):
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

### Sorting points of a grid, Fig. 11

In [ ]:
grid_num = 10

In [ ]:
np.random.seed(10)
pop_front = create_uniform_grid(num=grid_num)
calc_fitness(pop_front)
fronts_front = sortNondominated(pop_front, len(pop_front))

In [ ]:
np.random.seed(10)
pop_po = create_uniform_grid(num=grid_num)
calc_fitness(pop_po)
fronts_po = sortPO(pop_front, len(pop_front))

In [ ]:
num_fronts = 7

Sorting by Pareto fronts

In [ ]:
figure(figsize=(4, 4), dpi=80)
pop_fitness = np.array([el.fitness.values for el in pop_front])
plt.scatter(pop_fitness[:, 0], pop_fitness[:, 1], s=5, c="grey", marker='.')

marker_arr = ['o', '<', '+', 'v', '_', 's', '1', '*', 'd', 'x']
for i in range(0, num_fronts):
    pop_fitness = np.array([el.fitness.values for el in fronts_front[i]])
    plt.scatter(pop_fitness[:, 0], pop_fitness[:, 1], s=50, c="k", 
                marker='${}$'.format(i)
                #marker=marker_arr[i]
               )
    if i < 6:
        plt.plot(pop_fitness[0], np.flip(pop_fitness[0]), c='k', lw=0.5)
        
plt.xlabel('$x_1$') 
plt.ylabel('$x_2$') 

plt.xlim([-0.05, 0.65])
plt.ylim([-0.05, 0.65])

#tikzplotlib.save('sorting_grid_front.tex') 
#plt.savefig('sorting_grid_front.png')

Sorting by $k$-Pareto optimality

In [ ]:
def plot_hyp(x, a, b, c):
    plt.plot(x, a/(x-b) - c, c='k', lw=0.5)

In [ ]:
figure(figsize=(4, 4), dpi=80)
pop_fitness = np.array([el.fitness.values for el in pop_po])
plt.scatter(pop_fitness[:, 0], pop_fitness[:, 1], s=5, c="grey", marker='.')

marker_arr = ['o', '<', '+', 'v', '_', 's', '1', '*', 'd', 'x']
for i in range(0, num_fronts):
    pop_fitness = np.array([el.fitness.values for el in fronts_po[i]])
    plt.scatter(pop_fitness[:, 0], pop_fitness[:, 1], s=50, c="k", 
                marker='${}$'.format(i)
                #marker=marker_arr[i]
               )
    
# some tweaking to show hyperbola-like shapes
x = np.arange(0.01, 0.1, 0.001)  
plot_hyp(x, a=0.001, b=0, c=0)

x = np.arange(0.01, 0.2, 0.001)
plot_hyp(x, a=0.005, b=-0.015, c=0.015)
    
x = np.arange(0.01, 0.3, 0.001)
plot_hyp(x, a=0.02, b=-0.05, c=0.05)

x = np.arange(0.005, 0.4, 0.001)
plot_hyp(x, a=0.03, b=-0.06, c=0.06)

x = np.arange(0.008, 0.5, 0.001)
plot_hyp(x, a=0.07, b=-0.11, c=0.11)

plt.xlabel('$x_1$') 
plt.ylabel('$x_2$') 
plt.xlim([-0.05, 0.65])
plt.ylim([-0.05, 0.65])

#tikzplotlib.save('sorting_grid_po.tex') 
#plt.savefig('sorting_grid_po.png')

### Sorting uniformly distributed random points, Fig. 13

In [ ]:
pop_size = 500

In [ ]:
np.random.seed(10)
pop_front = toolbox.population(n=pop_size)
calc_fitness(pop_front)
fronts_front = sortNondominated(pop_front, len(pop_front))
print('Sorting by fronts, number of fronts = {}'.format(len(fronts_front)))

In [ ]:
np.random.seed(10)
pop_po = toolbox.population(n=pop_size)
calc_fitness(pop_po)
fronts_po = sortPO(pop_front, len(pop_front))
print('Sorting by po, number of fronts = {}'.format(len(fronts_po)))

In [ ]:
num_fronts = 10

Sorting by Pareto fronts

In [ ]:
figure(figsize=(4, 4), dpi=80)
# plotting all points
pop_fitness = np.array([el.fitness.values for el in pop_front])
plt.scatter(pop_fitness[:, 0], pop_fitness[:, 1], s=5, c="grey", marker='.')

# plotting first num_fronts fronts
for i in range(0, num_fronts):
    pop_fitness = np.array([el.fitness.values for el in fronts_front[i]])
    plt.scatter(pop_fitness[:, 0], pop_fitness[:, 1], s=10, c="k")
        
plt.xlabel('$x_1$') 
plt.ylabel('$x_2$') 

#tikzplotlib.save('sorting_uniform_front.tex') 

Sorting by $k$-Pareto optimality

In [ ]:
figure(figsize=(4, 4), dpi=80)
# plotting all points
pop_fitness = np.array([el.fitness.values for el in pop_po])
plt.scatter(pop_fitness[:, 0], pop_fitness[:, 1], s=5, c="grey", marker='.')

# plotting first num_fronts fronts
for i in range(0, num_fronts):
    pop_fitness = np.array([el.fitness.values for el in fronts_po[i]])
    plt.scatter(pop_fitness[:, 0], pop_fitness[:, 1], s=10, c="k")
        
plt.xlabel('$x_1$') 
plt.ylabel('$x_2$') 

#tikzplotlib.save('sorting_uniform_po.tex') 

# Wealthy and non-wealth states, Fig. 16

Check `wealthy_countries/README.md` for data and analysis scripts.

In [ ]:
import pandas as pd

In [ ]:
country_data = pd.read_csv("wealthy_countries/output.csv")
country_data.head()

In [ ]:
ax = country_data[~country_data['is_poor']].plot(
    kind='scatter',
    x='gni',
    y='hdi',
    c='k',
    grid=True,
)

ax = country_data[country_data['is_poor']].plot(
    kind='scatter',
    x='gni',
    y='hdi',
    c='grey',
    grid=True,
    ax=ax,
)

ax.set_xlabel('Gross National Income (GNI), \$')
ax.set_ylabel('Education  and  Health  Indicator')
ax.legend(['wealthy', 'non-wealthy'])

#tikzplotlib.save('wealth_indicator.tex') 
#plt.savefig('wealth_indicator.png')